# Chap 5

In [1]:
sealed trait Stream[+A]{
  def cons[A](hd: => A, tl: => Stream[A]): Stream[A] = {
    lazy val head = hd
    lazy val tail = tl
    Cons(() => head, () => tail)
  }
  
  def empty[A]: Stream[A] = Empty
  
  def headOption: Option[A] = this match {
    case Empty => None
    case Cons(h, t) => Some(h())
  }
  
  def foldRight[B](z: => B)(f: (A, => B) => B): B = this match {
    case Cons(h, t) => f(h(), t().foldRight(z)(f))
    case _ => z
  }
  
  def exists_(p: A => Boolean): Boolean = this match {
    case Cons(h, t) => p(h()) || t().exists(p)
    case _ => false
  }
  
  def exists(p: A => Boolean): Boolean = 
    foldRight(false)((a, b) => p(a) || b)
  
  // Exercise 5.1
  def toListRec: List[A] = this match {
    case Cons(h, t) => h() :: t().toListRec
    case Empty => Nil
  }
  def toList: List[A] = {
    @annotation.tailrec
    def go(st: Stream[A], acc: List[A]): List[A] = st match {
      case Cons(h, t) => go(t(), h() :: acc)
      case Empty => acc
    }
    go(this, List()).reverse
  }
  
  // Exercise 5.2
  def take(n: Int): Stream[A] = this match {
    case Cons(h, t) if (n > 1) => cons(h(), t().take(n-1))
    case Cons(h, t) if (n == 1) => cons(h(), empty)
    case _ => empty
  }
  def drop(n: Int): Stream[A] = this match {
    case Cons(h, t) if (n > 0) => t().drop(n-1)
    case _ => this
  }
  
  // Exercise 5.3
  def takeWhile(p: A => Boolean): Stream[A] = this match{
    case Cons(h, t) if (p(h())) => cons(h(), t().takeWhile(p))
    case _ => empty
  }
  
  
  // Exercise 5.4
  def forAll(p: A => Boolean): Boolean =
    foldRight(true)((a, b) => p(a) && b)
  
  // Exercise 5.5
  def takeWhile_(p: A => Boolean): Stream[A] =
    foldRight(empty[A])((a, b) => if(p(a)) cons(a, b) else empty)
  
  // Exercise 5.6
  def headOption_ : Option[A] = 
    foldRight(None: Option[A])((a, b) => Some(a))
  
  // Exercise 5.7
  def map[B](f: A => B): Stream[B] =
    foldRight(empty[B])((a, b) => cons(f(a), b))
  
  def filter[B](f: A => Boolean): Stream[A] =
    foldRight(empty[A])((a, b) => if(f(a)) cons(a, b) else b)
  
  def append[B>:A](e: => Stream[B]): Stream[B] =
    foldRight(e)((a, b) => cons(a, b))
  
  def flatMap[B](f: A => Stream[B]): Stream[B] =
    foldRight(empty[B])((a, b) => f(a).append(b))
  
  def repeat: Stream[A] = append(repeat)
}
case object Empty extends Stream[Nothing]
case class Cons[+A](h: () => A, t: () => Stream[A]) extends Stream[A]

object Stream {
  def cons[A](hd: => A, tl: => Stream[A]): Stream[A] = {
    lazy val head = hd
    lazy val tail = tl
    Cons(() => head, () => tail)
  }
  
  def empty[A]: Stream[A] = Empty
  
  def apply[A](as: A*): Stream[A] =
    if (as.isEmpty) empty else cons(as.head, apply(as.tail: _*))
  
}

defined trait Stream
defined object Empty
defined class Cons
defined object Stream

In [2]:
Stream(1,2,3).repeat.take(5).toList

res1: List[Int] = List(1, 2, 3, 1, 2)

In [3]:
Stream(1,2,3).toListRec

res2: List[Int] = List(1, 2, 3)

## 연습문제 5.1
Stream을 List로 변환하되 평가를 강제해서 REPL로 목록의 요소들을 볼 수 있게 하는 함수를 작성하라. 표준라이브러리에 있는 정규 List 형식으로 변환하면 된다. 이 함수와 Stream에 대해 작용하는 다른 함수들을 Stream 특질(trait)안에 넣어도 좋다. 
```scala
def toList: List[A]
```

In [ ]:
def toListRec: List[A] = this match {
  case Cons(h, t) => h() :: t().toListRec
  case Empty => Nil
}

def toList: List[A] = {
  @annotation.tailrec
  def go(st: Stream[A], acc: List[A]): List[A] = st match {
    case Cons(h, t) => go(t(), h() :: acc)
    case Empty => acc
  }
  go(this, List()).reverse
}

## 연습문제 5.2
Stream의 처음 n개의 요소를 돌려주는 함수 take(n)과 Stream의 처음 n개의 요소를 건너뛴 스트림을 돌려주는 drop(n)을 작성하라.

In [ ]:
// Exercise 5.2
def take(n: Int): Stream[A] = this match {
  case Cons(h, t) if (n > 1) => cons(h(), t().take(n-1))
  case Cons(h, t) if (n == 1) => cons(h(), empty)
  case _ => empty
}
def drop(n: Int): Stream[A] = this match {
  case Cons(h, t) if (n > 0) => t().drop(n-1)
  case _ => this
}

## 연습문제 5.3
Stream에서 주어진 술어를 만족하는 선행 요소들을 모두 돌려주는 함수 takeWhile을 작성하라.
```scala
def takeWhile(p: A => Boolean): Stream[A]
```

In [ ]:
// Exercise 5.3
def takeWhile(p: A => Boolean): Stream[A] = this match{
  case Cons(h, t) if (p(h())) => cons(h(), t().takeWhile(p))
  case _ => empty
}

In [16]:
Stream(1,2,3).take(2).toList

res15: List[Int] = List(1, 2)

## 연습문제 5.4
Stream의 모든 요소가 주어진 술어를 만족하는지 점검하는 forAll 함수를 구현하라. 만족하지 않는 값을 만나면 즉시 순회를 마쳐야 한다. 
```scala
def forAll(p: A => Boolean): Boolean
```

In [ ]:
// Exercise 5.4
def forAll(p: A => Boolean): Boolean =
  foldRight(true)((a, b) => p(a) && b)

## 연습문제 5.5
foldRight를 이용해서 takeWhile을 구현하라.

In [ ]:
def takeWhile(p: A => Boolean): Stream[A] =
  foldRight(empty)((a, b) => if(p(a)) cons(a, b))

## 연습문제 5.6 (hard)
foldRight를 이용해서 headOption을 구현하라.

In [ ]:
// Exercise 5.6
def headOption_ : Option[A] = 
  foldRight(None: Option[A])((a, b) => Some(a))

## 연습문제 5.7
foldRight를 이용해서 map, filter, append, flatMap을 구현하라. append 메서드는 자신의 인수에 대해 엄격하지 않아야 한다.

In [ ]:
// Exercise 5.7
def map[B](f: A => B): Stream[B] =
  foldRight(empty[B])((a, b) => cons(f(a), b))

def filter[B](f: A => Boolean): Stream[A] =
  foldRight(empty[A])((a, b) => if(f(a)) cons(a, b) else b)

def append[B>:A](e: => Stream[B]): Stream[B] =
  foldRight(e)((a, b) => cons(a, b))

def flatMap[B](f: A => Stream[B]): Stream[B] =
  foldRight(empty[B])((a, b) => f(a).append(b))

## infinite stream

In [2]:
val ones: Stream[Int] = Stream.cons(1, ones)
ones.take(5).toList
ones.exists(_ % 2 != 0)

ones: Stream[Int] = Cons(<function0>,<function0>)
res1_1: List[Int] = List(1, 1, 1, 1, 1)
res1_2: Boolean = true

In [3]:
ones.map(_ + 1).exists(_ % 2 ==0)
ones.takeWhile(_ == 1)
ones.forAll(_ != 1)

res2_0: Boolean = true
res2_1: Stream[Int] = Cons(<function0>,<function0>)
res2_2: Boolean = false

## 연습문제 5.8
ones를 조금 일반화해서, 주어진 값의 무한 Stream을 돌려주는 함수 constant를 구현하라.
```scala
def constant[A](a: A): Stream[A]
```

In [4]:
def constant[A](a: A): Stream[A] = {
  lazy val cs: Stream[A] = Stream.cons(a, cs)
  cs
}

defined function constant

## 연습문제 5.9
n에서 시작해서 n+1, n+2, 등으로 이어지는 무한 정수 스트림을 생성하는 함수를 작성하라.
```scala
def from(n: Int): Stream[Int]
```

In [5]:
def from(n: Int): Stream[Int] = {
  Stream.cons(n, from(n+1))
}

defined function from

## 연습문제 5.10
무한 피보나치 수 0, 1, 1, 2, 3, 5, 8, ...으로 이루어진 무한 스트림을 생성하는 함수 fibs를 작성하라. 

In [6]:
def fibs: Stream[Int] = {
  def go(before: Int, after: Int): Stream[Int] =
    Stream.cons(before, go(after, before+after))
  go(0,1)
}

defined function fibs

In [9]:
fibs.take(10).toList

res8: List[Int] = List(0, 1, 1, 2, 3, 5, 8, 13, 21, 34)

## 연습문제 5.11
좀더 일반화된 스트림 구축 함수 unfold를 작성하라. 이 함수는 초기 상태 하나와 다음 상태 및 다음 값(생성된 스트림 안의)을 산출하는 함수 하나를 받아야 한다.
```scala
def unfold[A, S](z: S)(f: S => Option[(A, S)]): Stream[A]
```

In [4]:
def unfold[A, S](z: S)(f: S => Option[(A, S)]): Stream[A] = f(z) match {
  case Some((a, s)) => Stream.cons(a, unfold(s)(f))
  case None => Stream.empty
}

defined function unfold

## 연습문제 5.12
unfold를 이용해서 fibs, from, constant, ones를 작성하라.

In [5]:
def ones : Stream[Int] =
  unfold(1)(x => Some((x, x)))

def constant[A](v: A): Stream[A] =
  unfold(v)(x => Some(x, x))

def from(n: Int): Stream[Int] =
  unfold(n)(x => Some(x, x+1))

def fibs: Stream[Int] =
  unfold((0, 1)) { case (b, a) => Some((b, (a, b+a))) }

defined function ones
defined function constant
defined function from
defined function fibs

In [6]:
from(4).take(5).toList

res5: List[Int] = List(4, 5, 6, 7, 8)

## 연습문제 5.13
unfold를 이용해서 map, take, takeWhile, zipWith(제3장 참고), zipAll을 구현하라. zipAll함수는 스트림에 요소가 더 있는 한 순회를 계속해야 한다. 각 스트림이 소진되었는지는 Option을 이용해서 지정한다.
```scala
def zipAll[B](s2: Stream[B]): Stream[(Option[A], Option[B])]
```

In [7]:
def map[A,B](st: Stream[A])(f: A => B): Stream[B] = 
  unfold(st){ 
    case Cons(h, t) => Some(f(h()), t())
    case _ => None
  }

def take[A](st: Stream[A])(n: Int): Stream[A] =
  unfold((st, n)){
    case (Cons(h, t), n) if (n > 1) => Some(h(), (t(), n-1))
    case (Cons(h, t), 1) => Some(h(), (Stream.empty[A], 0))
    case _ => None
  }

def takeWhile[A](st: Stream[A])(f: A => Boolean): Stream[A] =
  unfold(st){
    case Cons(h, t) if f(h()) => Some(h(), t())
    case _ => None
  }

def zipWith[A,B,C](s1: Stream[A], s2: Stream[B])(f: (A, B) => C): Stream[C] = 
  unfold((s1, s2)){
    case (Cons(h1, t1), Cons(h2, t2)) => Some(f(h1(), h2()), (t1(), t2()))
    case _ => None
  }

def zipAll[A,B](s1: Stream[A], s2: Stream[B]): Stream[(Option[A], Option[B])] =
  unfold((s1, s2)){
    case (Cons(h1, t1), Cons(h2, t2)) => Some((Some(h1()), Some(h2())), (t1(), t2()))
    case (Cons(h1, t1), Empty) => Some((Some(h1()), Option.empty[B]), (t1(), Stream.empty[B]))
    case (Empty, Cons(h2, t2)) => Some((Option.empty[A], Some(h2())), (Stream.empty[A], t2()))
    case _ => None
  }

defined function map
defined function take
defined function takeWhile
defined function zipWith
defined function zipAll

## 연습문제 5.14 (hard)
앞에서 작성한 함수들을 이용해서 startsWith를 구현하라. 이 함수는 한 Stream이 다른 한 Stream의 선행 순차열(prefix)인지 점검해야 한다. 예를 들어 Stream(1,2,3) startsWith Stream(1,2)는 true가 되어야 한다.
```scala
def startsWith[A](s: Stream[A]): Boolean
```

In [15]:
def startsWith[A](as: Stream[A], s: Stream[A]): Boolean =
  zipAll(as, s)((a, b) => a == b).forAll(_ == true)

cmd15.sc:2: cmd15Wrapper.this.cmd6.cmd0.wrapper.Stream[(Option[A], Option[A])] does not take parameters
  zipAll(as, s)((a, b) => a == b).forAll(_ == true)
               ^

: 

In [12]:
startsWith(Stream(1,2,3,4), Stream(2,3))

res11: Boolean = false

In [18]:
def startsWith[A](ss: Stream[A], s: Stream[A]): Boolean =
  zipAll(ss, s).takeWhile(_._2.isEmpty) forAll {
    case (h,h2) => h == h2 
  }

defined function startsWith

In [29]:
zipAll(Stream(1,2,3,4,5,6,7), Stream(1,2,3,4,5)).takeWhile(!_._2.isEmpty).toList

res28: List[(Option[Int], Option[Int])] = List(
  (Some(1), Some(1)),
  (Some(2), Some(2)),
  (Some(3), Some(3)),
  (Some(4), Some(4)),
  (Some(5), Some(5))
)

In [19]:
startsWith(Stream(1,2,3,4), Stream(1,2,3,4,5))

res18: Boolean = true

## 연습문제 5.15
unfold를 이용해서 tails를 구현하라. tails는 주어진 입력 Stream과 그 후행 순차열(suffix)들로 이루어진 스트림을 돌려준다. 예를 들어 Stream(1,2,3)에 대해 이 함수는 원래의 Stream(Stream(1,2,3), Stream(2,3), Stream(3), Stream())을 돌려주어야 한다.
```scala
def tails: Stream[Stream[A]]
```

In [36]:
def tails[A](st: Stream[A]): Stream[Stream[A]] = 
  unfold(st){
    case Cons(h, t) => Some((Cons(h,t), t()))
    case Empty => None
  }

defined function tails

In [37]:
tails(Stream(1,2,3)).toList.map(_.toList)

res36: List[List[Int]] = List(List(1, 2, 3), List(2, 3), List(3))

## 연습문제 5.16 (hard)
tails를 일반화한 scanRight 함수를 작성하라. 이 함수는 중간 결과들의 스트림을 돌려주는 foldRight와 비슷하다. 예:
```bash
scala> Stream(1,2,3).scanRight(0)(_ + _).toList
res0: List[Int] = List(6,5,3,0)
```
이 예는 표현식 List(1+2+3+0, 2+3+0, 3+0, 0)과 동등해야 한다. 독자의 구현은 중간 결과들을 재사용해야 한다. 즉, 요소가 n개인 Stream을 훑는 데 걸리는 시간이 항상 n에 선형 비례해야 한다. 이 함수를 unfold를 이용해서 구현할 수 있을까? 있다면 어떻게? 없다면 왜 그럴까? 이 함수를 앞에서 작성한 다른 어떤 함수로 구현할 수는 있을까?